In [ ]:
#installing nessesary packages

install.packages("rpart")
install.packages("rpart.plot")
install.packages("caTools")
library(caTools)
library(rpart)
library(rpart.plot)

In [ ]:
#just to getting nessasary data after pre processing
original_data <- read.csv("./dataset.csv", header = TRUE)
show(original_data)



: 

In [ ]:
# to create the heat map so can get idea of corelation between the features

corr_dataset <- dataset

numeric_data <- corr_dataset %>%
  select_if(is.numeric)

cor_matrix <- cor(numeric_data)

par(mar = c(2, 2, 2, 2))  # Adjust the margins if needed
corrplot(cor_matrix, method = "color", order="AOE", addCoef.col = "black", tl.col = "black", tl.srt = 90, number.cex = 0.4)

: 

In [ ]:
#data to be processed
data_set <- read.csv("./dataset.csv", header = TRUE)

#cleaning dataset according to the heat map 

columns_to_remove <- c("Over18",
                       "EmployeeCount",
                       "BusinessTravel",
                       "EmployeeNumber" ,
                       "StandardHours" ,
                       "YearsWithCurrManager" ,
                       "Department",
                       "Age",
                       "YearsAtCompany",
                       "YearsSinceLastPromotion",
                       "YearsInCurrentRole",
                       "JobLevel",
                       "YearsInCurrentRole",
                       "NumCompaniesWorked",
                       "Gender"
                       )

data_set <- data_set[, -which(names(data_set) %in% columns_to_remove)]





: 

In [ ]:
# Initialize a vector to store accuracies
accuracies <- numeric(10)

: 

In [ ]:
#process of desicion tree model for 10 times

for (i in 1:10) {
  
  # Set seed for each iteration (optional, depends on your use case)
  set.seed(120+ i*i)
  
  split_ratio <- sample.split(data_set, SplitRatio = 0.7)
  
  train_data <-  subset(data_set, split_ratio=="TRUE")
  test_data <- subset(data_set, split_ratio=="FALSE")
  
  
  head(test_data)
  
  decision_tree <- rpart(Attrition ~ ., data = train_data, method = "class")
  
  predictions <- predict(decision_tree, newdata = test_data, type = "class")
  
  confusion_matrix <- table(predictions, test_data$Attrition)
  
  dim(confusion_matrix)
  
  accuracy <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
  
  # Store the accuracy in the vector
  accuracies[i] <- accuracy
  
  print(paste("Accuracy for iteration", i, ":", accuracy))
}


: 

In [ ]:
#to print the accuracy of the model for 10 times

average_accuracy <- mean(accuracies)

print(paste("Average Accuracy:", average_accuracy))

